# 1 Import Libraries



In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import to_datetime
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

# 2 Load Data

In [2]:
# Mengambil file
file_path = 'household_power_consumption.txt'

# Mengambil data dari file
# Menggabungkan kolom Date & Time
df = pd.read_csv(file_path, sep=';', dayfirst=True,
                 infer_datetime_format=True,
                 low_memory=False, na_values=['nan', '?'])

df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


# 3 Preprocess Data

In [3]:
# Menggabungkan kolom Date & Time
df['Datetime'] = pd.to_datetime(df['Date'] + " " + df['Time'])

df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Datetime
0,16/12/2006,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0,2006-12-16 17:24:00
1,16/12/2006,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0,2006-12-16 17:25:00
2,16/12/2006,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0,2006-12-16 17:26:00
3,16/12/2006,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0,2006-12-16 17:27:00
4,16/12/2006,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0,2006-12-16 17:28:00


In [4]:
df.to_csv('house_consumption_melted.csv', index=False)

# 4 Model Build

## Process Melted Dataset

In [5]:
# Ambil Data
df = pd.read_csv('house_consumption_melted.csv', sep=',', infer_datetime_format=True, header=0)

# Convert 'Datetime' column to datetime type
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Set 'Datetime' as the index
df.set_index('Datetime', inplace=True)

# Group by minute and then resample into 1-hour frequency while summing 'Sub_metering_3'
df_agg = df.resample('1H').agg({'Sub_metering_3': 'sum'})

df_agg

,Sub_metering_3
Datetime,
2006-12-16 17:00:00,607.0
2006-12-16 18:00:00,1012.0
2006-12-16 19:00:00,1001.0
2006-12-16 20:00:00,1007.0
2006-12-16 21:00:00,1033.0
...,...
2010-12-11 19:00:00,788.0
2010-12-11 20:00:00,604.0
2010-12-11 21:00:00,0.0


## Select the Relevant Data

In [6]:
# Select relevant columns
data = df_agg[['Sub_metering_3']]

# Find the maximum value
max_value = data['Sub_metering_3'].max()

# Find the minimum value
min_value = data['Sub_metering_3'].min()
# Print the result
print(data)
print(f"Maximum value: {max_value}")
print(f"Minimum value: {min_value}")

                     Sub_metering_3
Datetime                           
2006-12-16 17:00:00           607.0
2006-12-16 18:00:00          1012.0
2006-12-16 19:00:00          1001.0
2006-12-16 20:00:00          1007.0
2006-12-16 21:00:00          1033.0
...                             ...
2010-12-11 19:00:00           788.0
2010-12-11 20:00:00           604.0
2010-12-11 21:00:00             0.0
2010-12-11 22:00:00             0.0
2010-12-11 23:00:00             0.0

[34951 rows x 1 columns]
Maximum value: 1293.0
Minimum value: 0.0


## Normalize Data

In [7]:
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data)

print(data_normalized)

[[0.46945089]
 [0.78267595]
 [0.7741686 ]
 ...
 [0.        ]
 [0.        ]
 [0.        ]]


## Splits The Data

In [8]:
split_time = int(len(data) * 0.5)
train_data = data_normalized[:split_time]
train_target = data_normalized[split_time:]

## Windowed The Dataset

In [9]:
BATCH_SIZE = 32
N_PAST = 24
N_FUTURE = 24
SHIFT = 1

In [10]:
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(n_past + n_future))
    dataset = dataset.shuffle(1000)
    dataset = dataset.map(lambda window: (window[:-n_past], window[-n_past:, :1]))
    return dataset.batch(batch_size).prefetch(1)

In [11]:
# Code to create windowed train and validation datasets.
train_set = windowed_dataset(train_data, BATCH_SIZE, N_PAST, N_FUTURE, SHIFT)
valid_set = windowed_dataset(train_target, BATCH_SIZE, N_PAST, N_FUTURE, SHIFT)

## Build The Model

In [26]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(N_FUTURE,1)),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(20, activation="relu"),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(N_FUTURE)
])

# Summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 24, 64)            16896     
                                                                 
 lstm_3 (LSTM)               (None, 64)                33024     
                                                                 
 dense_3 (Dense)             (None, 20)                1300      
                                                                 
 dense_4 (Dense)             (None, 10)                210       
                                                                 
 dense_5 (Dense)             (None, 24)                264       
                                                                 
Total params: 51694 (201.93 KB)
Trainable params: 51694 (201.93 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Compile the Model

In [27]:
model.compile(optimizer='adam', loss='mae', metrics=["mae"])

### Callback Function

In [28]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('mae') < 0.25:
            print('MAE terpenuhi')
            self.model.stop_training = True

### Train the Model

In [29]:
model.fit(train_set, validation_data=valid_set, epochs=300, batch_size=BATCH_SIZE, callbacks=[myCallback()])

Epoch 1/300
545/545 [==============================] - 31s 48ms/step - loss: 0.2607 - mae: 0.2607 - val_loss: 0.2720 - val_mae: 0.2720
Epoch 2/300
545/545 [==============================] - 29s 52ms/step - loss: 0.2579 - mae: 0.2579 - val_loss: 0.2576 - val_mae: 0.2576
Epoch 3/300
545/545 [==============================] - 28s 51ms/step - loss: 0.2467 - mae: 0.2467 - val_loss: 0.2522 - val_mae: 0.2522


### Test the Model

In [30]:
# Make predictions on test data
predictions = model.predict(train_set)
predictions

545/545 [==============================] - 19s 16ms/step


array([[0.04079525, 0.04678063, 0.02120738, ..., 0.06277222, 0.07368964,
        0.06584389],
       [0.07253189, 0.35793713, 0.70979476, ..., 0.00187754, 0.0061076 ,
        0.0358789 ],
       [0.04465834, 0.03059525, 0.02508501, ..., 0.14731994, 0.0943978 ,
        0.07457177],
       ...,
       [0.18642548, 0.314005  , 0.27950758, ..., 0.01727192, 0.09987184,
        0.13517512],
       [0.03217737, 0.22218914, 0.39796594, ..., 0.0462469 , 0.02859987,
        0.03144878],
       [0.27856052, 0.096139  , 0.07927296, ..., 0.48636436, 0.3557917 ,
        0.31306672]], dtype=float32)

In [31]:
# Inverse transform predictions to original scale
predictions = scaler.inverse_transform(predictions)
data_normalized_inverse = scaler.inverse_transform(data_normalized[:split_time])

In [35]:
print(predictions)

[[ 52.74826    60.48735    27.421139  ...  81.16448    95.2807
   85.13615  ]
 [ 93.78374   462.8127    917.76465   ...   2.4276578   7.897125
   46.391415 ]
 [ 57.74323    39.559658   32.43492   ... 190.48468   122.05635
   96.421295 ]
 ...
 [241.04814   406.00845   361.4033    ...  22.332594  129.1343
  174.78143  ]
 [ 41.605343  287.29056   514.56995   ...  59.797237   36.979626
   40.66327  ]
 [360.17874   124.307724  102.49993   ... 628.86914   460.03867
  404.79526  ]]


In [36]:
print(data_normalized_inverse)

[[ 607.]
 [1012.]
 [1001.]
 ...
 [ 659.]
 [   0.]
 [ 356.]]


# 5 Generate Save Model

## Save the Model

In [33]:
run_model = tf.function(lambda x: model(x))
STEPS = 1
INPUT_SIZE = 24
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([STEPS, INPUT_SIZE], model.inputs[0].dtype))

# model directory.
MODEL_DIR = "model_kWh_5"
model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
tflite_model = converter.convert()

## Convert the Model to TFLITE

In [34]:
import pathlib
tflite_model_file = pathlib.Path('model_kWh_5.tflite')
tflite_model_file.write_bytes(tflite_model)

213228